<a href="https://colab.research.google.com/github/andreeas26/TSM_UNetSegmentation/blob/master/evaluate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preparing the setup

In [ ]:
#@title Mount drive content
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')
%cd /content/drive/My\ Drive/Work/TSM

In [ ]:
import pathlib as pt
import os
import yaml
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import keras
import tensorflow as tf
from datetime import datetime
from tqdm import tqdm
from keras.preprocessing.image import img_to_array
from keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping
from keras.models import load_model
from keras import optimizers

import generators as gen
import custom_metrics as cm
from UNetModel import UNetModel

print("Keras", keras.__version__)
print("Tensorflow", tf.__version__)

### Set useful paths


In [ ]:
# paths for the data
ROOT_PATH    = pt.Path("/content/drive/My Drive/Work/TSM")
DATASET_PATH = ROOT_PATH/"dataset"
IMGS_DIR     = "images"
MASKS_DIR    = "masks"
CONFIG_PATH  = ROOT_PATH/"config_evaluate.yaml"
IMGS_CSV     = DATASET_PATH/"images.csv"
MASKS_CSV    = DATASET_PATH/"masks.csv"

# paths for evaluation
RUN_TIME         = '2020_08_28-14_40'
TRAIN_PATH       = ROOT_PATH/"trainings"/RUN_TIME
CHECKPOINTS_PATH = TRAIN_PATH/"checkpoints"
MODEL_PATH       = CHECKPOINTS_PATH/"unet_weights-95-0.50641.hdf5"
EPOCH_NR         = str(MODEL_PATH.stem).split('-')[1]
PRED_PATH        = TRAIN_PATH/"predictions"/EPOCH_NR
GRAPHS_PATH      = TRAIN_PATH/"graphs"
if not PRED_PATH.exists():
  os.makedirs(PRED_PATH)

## Evaluating U-Net


In [ ]:
with open(CONFIG_PATH) as fp:
  config = yaml.safe_load(fp)
config

In [ ]:
imgs_df  = pd.read_csv(IMGS_CSV)
masks_df = pd.read_csv(MASKS_CSV)

assert len(imgs_df) == len(masks_df)

In [ ]:
test_imgs_df  = imgs_df.loc[imgs_df['subset'] == "test", :]
test_masks_df = masks_df.loc[masks_df['subset'] == "test", :]
print("[INFO] Test size {}".format(len(test_imgs_df)))

### Load a model

In [ ]:
custom_objects = {"dice_coef_loss": cm.dice_coef_loss, "dice_coef": cm.dice_coef}
model = load_model(MODEL_PATH, custom_objects=custom_objects)
print("[INFO] Loaded model from {}".format(MODEL_PATH))

In [ ]:
!pip install medpy
from medpy.metric import dc, jc

In [ ]:
def dice_coef(y_true, y_pred):
    intersec = y_true*y_pred
    union = y_true+y_pred
    if intersec.sum() == 0:
        dice_coef = 0
    else:
        dice_coef = round(intersec.sum()*2/union.sum(), 2)
    return dice_coef

In [ ]:
score_df = pd.DataFrame(columns=["img_path", "mask_true_path", "mask_pred_path", "dice_score", "iou_score"])

if config["n_channels"] == 1:
    flag = cv2.IMREAD_GRAYSCALE
else:
    flag = cv2.IMREAD_COLOR

with tqdm(desc="Evaluating", unit="image", total=len(test_imgs_df), leave=True) as pbar:
  for _imgs, _masks in zip(test_imgs_df.iterrows(), test_masks_df.iterrows()):
      img_idx, img_row = _imgs
      mask_idx, mask_row = _masks

      img_name = pt.Path(img_row['file_path']).name
      mask_name = pt.Path(mask_row['file_path']).name
      assert img_name == mask_name

      img = cv2.imread(img_row['file_path'], flag)
      img = cv2.resize(img, dsize=tuple(config['target_size']))

      img = img * 1. / 255
      img = img_to_array(img)
      img = np.expand_dims(img, axis=0)
      pred_mask = model.predict(img, batch_size=1)
      pred_mask[pred_mask < config["threshold"]] = 0.0
      pred_mask[pred_mask > config["threshold"]] = 1.0
      pred_mask = np.squeeze(pred_mask)

      true_mask = cv2.imread(mask_row['file_path'], cv2.IMREAD_GRAYSCALE)
      true_mask = cv2.resize(true_mask, dsize=tuple(config['target_size']))

      dice_score = dc(pred_mask, true_mask)
      iou_score = cm.iou_score(true_mask, pred_mask)

      # print(dice_score, cm.dice_score(true_mask, pred_mask), dice_coef(true_mask, pred_mask))
      # print(iou_score, jc(pred_mask, true_mask))
      save_to = str(PRED_PATH/img_name)
      # print(save_to)
      cv2.imwrite(save_to, pred_mask * 255)
      score_df = score_df.append({"img_path": img_row['file_path'], 
                                  "mask_true_path": mask_row['file_path'], 
                                  "mask_pred_path": save_to,
                                  "dice_score": dice_score, 
                                  "iou_score": iou_score},
                                  ignore_index=True)

      # plt.imshow(pred_mask, cmap='gray')
      # plt.show()

      pbar.update()
print(score_df.describe())


In [ ]:
pred_csv = PRED_PATH/"predictions_thresh_{}.csv".format(config["threshold"])
print("[INFO] Saving predictions scores in {}.".format(pred_csv))
score_df.to_csv(pred_csv, index=False)

### Plot histograms

In [ ]:
def plot_histogram_scores(scores, num_bins=50, metric='Dice Index', facecolor='blue', save_to=None, fig_filename='histogram'):
    """
    Plot histogram of the Dice and IoU scores on the test set
    :param scores:
    :param num_bins:
    :param metric:
    :param facecolor:
    :param save_to:
    :param fig_filename:
    :return:
    """

    fig, ax = plt.subplots(figsize=(7, 5))
    # the histogram of the data
    _ = ax.hist(scores, bins=num_bins, density=True, facecolor=facecolor, alpha=0.7)
    ax.set_xlabel(metric)
    ax.set_ylabel('Frequency')
    ax.set_title('Histogram of {} values'.format(metric))
    ax.grid()
    # Tweak spacing to prevent clipping of ylabel
    fig.tight_layout()

    if save_to:
        if not save_to.exists():
            os.makedirs(str(save_to))

        plt.savefig(save_to/'{}_{}.png'.format(fig_filename, metric), dpi=300, bbox_inches='tight', pad_inches=0)
    else:
        plt.show()


def create_histograms(pred_csv_path, graphs_path):
    """
    Reads the predictions for each image in the test set and creates the
    histogram of the values
    :param pred_csv_path:
    :param graphs_path:
    :return:
    """
    print("[INFO] Plot histogram for {} in {}".format(pred_csv_path, graphs_path))

    pred_df = pd.read_csv(pred_csv_path)
    plot_histogram_scores(scores=pred_df["dice_score"].values,
                          metric="Dice Index",
                          save_to=graphs_path,
                          fig_filename="hist_dice")

    plot_histogram_scores(scores=pred_df["iou_score"].values,
                          metric="IoU",
                          save_to=graphs_path,
                          facecolor='green',
                          fig_filename="hist_iou")

In [ ]:
create_histograms(pred_csv, GRAPHS_PATH)